In [ ]:
import sys
import time
import torch
from pathlib import Path
import matplotlib.pyplot as plt
import os

print(os.getcwd())
os.chdir('..')
print(os.getcwd())

In [ ]:
# coming from the MonoRec directory
home = str(Path.home())
sys.path.append(os.path.join(home, "deepdriving/MonoRec"))
from data_loader.kitti_odometry_dataset import KittiOdometryDataset
from model.monorec.monorec_model import MonoRecModel
from utils import unsqueezer, map_fn, to

In [ ]:
target_image_size = (256, 512)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

kitti_path = os.path.join(home, "KITTI/dataset")

dataset = KittiOdometryDataset(kitti_path, 
                               frame_count=2,
                               sequences=["04"], 
                               depth_folder="image_depth_annotated", 
                               target_image_size=target_image_size, 
                               max_length=None,
                               offset_d=0,
                               use_color=True,
                               use_dso_poses=False,
                               lidar_depth=True, 
                               dso_depth=False,
                               annotated_lidar=True,
                               return_stereo=False,
                               return_mvobj_mask=False,
                               use_index_mask=())

checkpoint_location = os.path.join(home, "deepdriving/MonoRec/saved/checkpoints/monorec_depth_ref.pth")
inv_depth_min_max = [0.33, 0.0025]
print("Initializing model...")
monorec_model = MonoRecModel(checkpoint_location=checkpoint_location, 
                             inv_depth_min_max=inv_depth_min_max)

monorec_model.to(device)
monorec_model.eval()

print('YAY! Im ready')

In [ ]:
for i in range(1101):
    
    print("Fetching data...")
    index = i

    batch, depth = dataset.__getitem__(index)
    batch = map_fn(batch, unsqueezer)
    depth = map_fn(depth, unsqueezer)

    batch = to(batch, device)

    print("Starting inference...")
    s = time.time()
    with torch.no_grad():
        data = monorec_model(batch)

    prediction = data["result"][0, 0].cpu()
    mask = data["cv_mask"][0, 0].cpu()
    depth = depth[0, 0].cpu()

    e = time.time()
    print(f"Inference took {e - s}s")

    plt.imsave("depth"+str(i)+".png", prediction.detach().squeeze())
    plt.imsave("mask"+str(i)+".png", mask.detach().squeeze())
    plt.imsave("kf"+str(i)+".png", batch["keyframe"][0].permute(1, 2, 0).cpu().numpy() + 0.5)

    plt.title(f"MonoRec (took {e - s}s)")
    plt.imshow(prediction.detach().squeeze(), vmin=1 / 80, vmax=1 / 5)
    plt.show()
    plt.imshow(mask.detach().squeeze())
    plt.show()
    plt.imshow(batch["keyframe"][0].permute(1, 2, 0).cpu() + .5)
    plt.show()
    time.sleep(2)
